#### *Bringing all necessary libraries*

In [1]:
from __future__ import print_function, division

import torch
import argparse
import json 
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
from functools import partial 
import time
import os
import random
import copy
import pdb 
from tqdm import tqdm
from _utils import accuracy

#### *Applying Horizontal Flip to data as augmentation technique*

In [2]:
def transfrom_data():
    data_transforms = {
    'train': transforms.Compose([
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(), 
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                            std=[0.229, 0.224, 0.225])
        ]),
    'valid': transforms.Compose([
        transforms.ToTensor(), 
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                            std=[0.229, 0.224, 0.225])
        ])
    }

    return data_transforms

#### *Loading data by batches*

In [3]:
def load_data(batch_size, num_workers):
    print("Start loading data")
    data_dir = '../data'
    image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), transfrom_data()[x]) for x in ['train', 'valid']}
    dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size, shuffle=True, num_workers=num_workers) \
                    for x in ['train', 'valid']}
    class_names = image_datasets['train'].classes
    dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'valid']}
    print("Dataset sizes: Train {} Val {}".format(dataset_sizes['train'], dataset_sizes['valid']))
    print("Number of classes: Train {} Val {}".format(len(image_datasets['train'].classes), len(image_datasets['valid'].classes)))

    return dataloaders, class_names, dataset_sizes

#### *Loading the model with adjusting number of classes to 196 for pretrained ImageNet model*

In [4]:
def load_model(class_names):
    model = models.densenet161(pretrained=True)
    num_ftrs = model.classifier.in_features
    model.classifier = nn.Linear(num_ftrs, len(class_names))

    model = torch.nn.DataParallel(model.cuda(), device_ids=[0])

    return model

#### *Training the model*

In [5]:
def train_model(dataloaders, model, dataset_sizes, criterion, optimizer, scheduler, num_epochs, save_dir, f):
    since = time.time()
    
    # best_model_wts = copy.deepcopy(model.state_dict())
    best_val_top1_acc = 0.0
    best_val_epoch = -1 
    final_val_top5_acc = 0.0
    best_test_top1_acc = 0.0
    best_test_epoch = -1
    final_test_top5_acc = 0.0 

    for epoch in range(num_epochs):  # loop over the dataset multiple times
        for phase in ['train', 'valid']:
            if phase == 'train':
                scheduler.step()
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            top1_running_corrects = 0
            top5_running_corrects = 0

            it = tqdm(range(len(dataloaders[phase])), desc="Epoch {}/{}, Split {}".format(epoch, num_epochs - 1, phase), ncols=0)
            data_iter = iter(dataloaders[phase])
            for niter in it:
                inputs, labels = data_iter.next()
                inputs = inputs.cuda()
                labels = labels.cuda()

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    prec1, prec5 = accuracy(outputs, labels, topk=(1,5))
                    loss = criterion(outputs, labels)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                training_loss = loss.item()
                running_loss += loss.item() * inputs.size(0)
                top1_running_corrects += prec1[0]
                top5_running_corrects += prec5[0]

            epoch_loss = running_loss / dataset_sizes[phase]
            top1_epoch_acc = float(top1_running_corrects) / dataset_sizes[phase]
            top5_epoch_acc = float(top5_running_corrects) / dataset_sizes[phase]
            print('{} Epoch Loss: {:.6f} Epoch top1 Acc: {:.6f} Epoch top5 Acc: {:.6f}\n'.format(phase, epoch_loss, top1_epoch_acc, top5_epoch_acc))
            with open(epoch_trace_f_dir, "a") as f:
                lr = optimizer.param_groups[0]['lr']
                f.write("{},{},{},{:e},{:e},{:e}\n".format(epoch,phase,lr,epoch_loss,top1_epoch_acc,top5_epoch_acc))

            if phase == 'valid' and top1_epoch_acc > best_val_top1_acc:
                print("Top1 val Acc improve from {:6f} --> {:6f}".format(best_val_top1_acc, top1_epoch_acc))
                best_val_top1_acc = top1_epoch_acc
                final_val_top5_acc = top5_epoch_acc
                best_val_epoch = epoch
                save_f_dir = os.path.join(save_dir, "best_val_model.ft")
                print("Saving best val model into {}...".format(save_f_dir))
                torch.save(model.state_dict(), save_f_dir)


        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best top1 val Acc: {:6f}'.format(best_val_top1_acc))
    print('Final top5 val Acc: {:6f}'.format(final_val_top5_acc))
    print('Best val model is saved at epoch # {}'.format(best_val_epoch))



if __name__=="__main__":

    dataloaders, class_names, dataset_sizes = load_data(16, 12)
    model= load_model(class_names)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adagrad(model.parameters(), lr=0.001)
    steps = 20
    scheduler = lr_scheduler.StepLR(optimizer, step_size=steps, gamma=0.1)
    num_epochs = 30

    save_dir = './outputs/'
    if not os.path.isdir(save_dir):
        os.makedirs(save_dir)
    epoch_trace_f_dir = os.path.join(save_dir, "trace.csv")
    with open(epoch_trace_f_dir, "w") as f:
        f.write("epoch,split,lr,loss,top1_acc,top5_acc\n")

    train_model(dataloaders, model, dataset_sizes, criterion, optimizer, scheduler, num_epochs, save_dir, f)

Start loading data
Dataset sizes: Train 13030 Val 1629
Number of classes: Train 196 Val 196


Epoch 0/29, Split train: 100% 815/815 [06:41<00:00,  2.36it/s]


train Epoch Loss: 3.458291 Epoch top1 Acc: 0.400614 Epoch top5 Acc: 0.647659



Epoch 0/29, Split valid: 100% 102/102 [00:21<00:00,  7.18it/s]


valid Epoch Loss: 2.028569 Epoch top1 Acc: 0.650092 Epoch top5 Acc: 0.918355

Top1 val Acc improve from 0.000000 --> 0.650092
Saving best val model into ./outputs/best_val_model.ft...



...

valid Epoch Loss: 0.427968 Epoch top1 Acc: 0.903008 Epoch top5 Acc: 0.985267




Epoch 29/29, Split train: 100% 815/815 [06:33<00:00,  2.53it/s]


train Epoch Loss: 0.014598 Epoch top1 Acc: 0.999079 Epoch top5 Acc: 1.000000



Epoch 29/29, Split valid: 100% 102/102 [00:21<00:00,  7.39it/s]


valid Epoch Loss: 0.427087 Epoch top1 Acc: 0.903622 Epoch top5 Acc: 0.984653


Training complete in 205m 13s
Best top1 val Acc: 0.906691
Final top5 val Acc: 0.984653
Best val model is saved at epoch # 22
